In [6]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-02 02:31:29--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-02 02:31:29 (13.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [9]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-07-02 02:57:32--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  3.16MB/s    in 0.2s    

2024-07-02 02:57:33 (3.16 MB/s) - ‘documents.json’ saved [658332/658332]



In [10]:
import minsearch

In [11]:
import json

In [12]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [14]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [20]:
documents[100]

{'text': 'In this section of the course, the 5432 port of pgsql is mapped to your computer’s 5432 port. Which means you can access the postgres database via pgcli directly from your computer.\nSo No, you don’t need to run it inside another container. Your local system will do.',
 'section': 'Module 1: Docker and Terraform',
 'question': 'PGCLI - INKhould we run pgcli inside another docker container?',
 'course': 'data-engineering-zoomcamp'}

In [23]:
index = minsearch.Index(
    text_fields=["question","text","section"], 
    keyword_fields=["course"]
)

In [24]:
index.fit(documents)

In [25]:
q = "can i join the course , if it has already started?"

In [30]:
boost={'questions': 3.0,'section': 0.5}
results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
    
)

In [31]:
results

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineerin

In [32]:
from openai import OpenAI

In [33]:
client = OpenAI()

In [68]:
q = "Will I get certification post completion?"

In [69]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role':'user','content':q}]
)

In [70]:
response.choices[0].message.content

'It depends on the course or program you are completing. Some courses or programs offer certification upon completion, while others do not. It is important to check the details of the course or program you are interested in to determine if certification is provided.'

In [71]:
prompt_template= """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [72]:
context=""

for doc in results:
    context = context + f"section : {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}|n|n"
    
    
    

In [73]:
prompt = prompt_template.format(question=q,context=context).strip()

In [74]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: Will I get certification post completion?

CONTEXT:
section : General course-related questions
question: Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.|n|nsection : General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're st

In [76]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role':'user','content': prompt}]
)

response.choices[0].message.content

'Yes, you will get certification upon completion of the course.'

In [113]:
def search(query):
    boost={'questions': 3.0,'section': 0.5}
    results = index.search(
    query=query,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5    
)
    return results

In [114]:
def build_prompt(query,search_results):
    prompt_template= """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context=""

    for doc in results:
        context = context + f"section : {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}|n|n"
    prompt = prompt_template.format(question=query,context=context).strip()

    return prompt
    
    
    
    

In [115]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role':'user','content': prompt}]
)

    return response.choices[0].message.content

In [120]:
def rag(query):
    query = query
    search_results  = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [124]:
rag("the course has already started , Can i Still enroll?")

'No, if the course has already started, you may not be able to enroll. It is best to contact the course instructor or coordinator for more information.'

In [126]:
from elasticsearch import Elasticsearch

In [128]:
#When hosted in CLoud , we need to provide that Url over here
es_client = Elasticsearch('http://localhost:9200')

In [129]:
es_client.info()

ObjectApiResponse({'name': 'fcb9f783d4f8', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'sUP13sPaQqmKdym9T6P0uw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [130]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index = index_name,body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [132]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [133]:
from tqdm.auto import tqdm

In [134]:
for doc in tqdm(documents):
    es_client.index(index = index_name,document=doc)
    

  0%|          | 0/948 [00:00<?, ?it/s]

In [135]:
query = "I just discovered teh course, Can i still join"

In [143]:
def elastic_search(query):

    search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
    response = es_client.search(index = index_name,body = search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit["_source"])
    return result_docs

In [144]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [142]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [145]:
def rag(query):
    query = query
    search_results  = elastic_search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [146]:
rag(query)

'Yes, you can still join the course. Make sure to follow the instructions provided in the FAQ database for setting up your environment and running the necessary commands to work with Kafka and other streaming technologies.'